# Musx Composers

This tutorial introduces musx <i>composers</i> -- python generators that execute in a scheduling queue to add musical data to a score.

Running this demo requires a jupyter kernel (runtime environment) that contains the musx package.  See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for directions on how to install musx in your environment.

In [ ]:
import musx
print(f"musx.version: {musx.version}")

In musx, a *composer* is a Python [generator](https://stackabuse.com/python-generators/) (function) that executes inside a time-based scheduler to add musical material to a score. A composer's implementation code often follows a common format as shown in this schematic:

<pre><b><font color='green'>def</font></b> <font color='blue'>composer</font>(score</font>, [...]):
    <i>[initializations...]</i>
    <b><font color='green'>for</font></b> ... :
        <i>[runtime statements...]</i>
        score.add(...)
        <b><font color='green'>yield</font></b> <i>waittime</i>
    <i>[finalizations...]</i>
</pre>


This template reveals that a composer generator provides at least one input parameter to receive the musical score, followed by any number of additional optional parameters according to the needs of the designer. The second line indicates that when the composer is called it can create whaever internal state variables it needs in place in order to add content to the score. The main body of the composer is typically represented by a python for loop that iterates to add musical events to the score and crucially yields back a *time delta* in seconds that informs the controlling scheduler how long the composer should wait before running again. If the composer stops yielding (i.e. the for loop has ended), or if the yield value is negative, then the scheduler will not insert the composer back into score and it will terminate.  After the for loop complete, the composer can optionally perform finalizations,  that "clean up" or take final actions just before the composer is terminated.

Multiple composer functions can run at the same time, and composer functions can *sprout* (add) other composer functions to the scheduler dynamically, as needed.
One way to think about a composer is that it represents a unique *time line* that runs in order to add material to a musical score. As such, it is always evaluated within the context of the score's *scheduler*, a time-based priority queue that represents the flow of musical time in the composition. The scheduler also provides several attributes that composers can use to find out the current runtime state of the score. 

Here is a simple first example of defining a composer function and running it in a score.

In [ ]:
score = musx.Score()

def simp(score, ident, length, rhythm):
    # initialization statements
    print(f'simp {ident}: Hiho! scoretime = {score.now}')
    # for loop executes actions and yields (waits) for its next runtime
    for _ in range(length):
        # our for loop just prints a message each time it runs,
        # it doesnt add anything to a score
        print(f'simp {ident}: Running... scoretime = {score.now}')
        # the yield statement is required, it tells the scheduler how long to
        # wait before running the for loop again.
        yield rhythm
    # post processing statements    
    print(f'simp {ident}: Tata! scoretime = {score.now}')

# Run simp in the scores scheduler, passing the function a string name and values for 
# its length and rhythm:
score.compose( simp(score, 'a', 3, 1.125) )

Here is an explanation of the example one line at a time:

------
```score = musx.Score()```

The variable `score` is set to an instance of a musx Score. A score contains a *scheduler*, which acts like a conductor: it manages musical time and ensures that composer functions execute at their correct times in the score.

-----
```def simp(score, ident, length, rhythm):```

A composer function can use any name but must define one or more input parameters. The first parameter will always receive the active score. The remaining parameters, if any, provide input (the initial state) for the composer to use.

-----
```    print(f'simp {ident}: Hiho! scoretime = {score.now}')```

An initialization statement.

-----
```    for _ in range(length):```

Music composition is a process that starts and ends. This can be reflected in different ways, one common method is for a composer to explicitly iterate using a for or while loop. Expressing the length of the iteration using a function parameter allows flexibility in how many times a composer can run.

-----
```        print(f'{ident}: scoretime = {score.now}')```

The print statement is the only real 'action' that this composer performs. Each time the composer executes the print statement displays the composer's identifier and the current score time.  The score time is managed by the score's scheduler, which also contains other dynamic state useful to the compositional process, including a score 'output destination', and the ability for a composer to add *new* composers to the composition as part of its workflow.

-----
```        yield rhythm```

A yield statement is required. As previously mentioned, a composer's yield value is the time increment, in seconds, until the composer's next run time. If the composer stops yielding, or if the yield value is negative, then it will not run again.

-----
```print(f'simp {ident}: Tata! scoretime = {score.now}')```

A finialization statement that executes after the composer's last yield. Even though the composer won't run again the finalization statements could still sprout new composers.

-----
```score.compose(simp(q, 'a', 10, .25))```

The scheduler's compose method starts the compositional process running. As we will see in the next example, it is possible to add multiple composers to the scheduler, and to give composers different *start times* in the composition.


### Multiple composers

In this example three separate instances of the simp() composer generate events at different rates to create three different time lines in the score. Note that at time 0, 1, 3 and 4 -- where the composers all share a common time point -- they are executing in the same order that they were added to the scheduler (a,b,c).

To run multiple composers in a Scheduler put them in a python list and pass the list to the compose() function:

In [ ]:
threesimps = [simp(score,'a',5,1), simp(score,'b',10,1/2), simp(score,'c',15,1/3)]
score.compose(threesimps)

### Specifying different start times

To add a composer to a score at a time later than time 0, specify the start time together with the composer as a two element list: [*start*, *composer*]. 

This example runs four composers, each composer starts 2 seconds later than the preceding one, sort of like a canon.

In [ ]:
score.compose( [[t*2, simp(score, t, 4, .25)] for t in range(4)] )

### Adding composers dynamically

A composer can create and add new composers dynamically, as part of its workflow while it is running. In this next example, each time `sprouter()` runs, it has a 50% probability of adding a new composer to the score with a start time 5 seconds into the future from the parent's current run time.

In [ ]:
def sprouter(score, length, rhythm):
    for i in range(length):
        print(f'sprouter: Running... scoretime = {score.now}')
        if musx.odds(.5):
            score.compose([5, simp(score, i, 5, .1)])
        yield rhythm

score.compose([sprouter(score, 5, 1), [10, sprouter(score, 3, 2)]])

## Compositional strategies with composers

Composers are very flexible and can be utilized in a variety of ways:

* A single composer can be called multiple times to create different versions of itself (e.g. same algorithm but with different initial states)
* There is no limit to the number of composers that are simultaneously running.
* A composer can at any point in its lifetime 'sprout' new composers to start at the current time or in the future.

## For more information...

The demos and tutorials directories contains numerous examples of using musx composers. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for more information.